# Packages

In [1]:
import geemap
import ee

from datetime import datetime

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

import os
os.getcwd()

'c:\\Users\\gilramolete\\OneDrive - UNIONBANK of the Philippines\\Documents 1\\Open Nighttime Lights'

# Load and inspect the DMSP-OLS nighttime lights Image Collection

In the context of remote sensing, the process of **making a composite** refers to taking a collection of spatially overlapping images and applying an aggregate function, also known as a "reducer" function, to these images to create a single composite image. For example, we might want to create an annual composite for a series of images by applying a reducer function, like calculating the median, to get a single image for the year containing the median values per pixel.

**Mosaicing** refers to the process of assembling different spatially located images from the same time period together to create a seamless single image.

Let's initialize a geemap object on the greater Washington, DC area.

In [2]:
# Set initial map parameters
center_lat = 38.9072
center_lon = -77.0369
zoom = 10

# Initialize map
map1 = geemap.Map(center = [center_lat, center_lon], zoom = zoom)
map1.add_basemap('SATELLITE')

map1.addLayerControl()
map1

Map(center=[38.9072, -77.0369], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

## Get DMSP-OLS Image Collection and review metadata and description

Recall the image collection for DMSP-OLS is located at: `NOAA/DMSP-OLS/NIGHTTIME_LIGHTS`

**What is the total number of images in this collection?** (It spans 1992-2013, but some years contain multiple images due to satellite overlap.)

In [3]:
dmsp = ee.ImageCollection('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS')

dmsp.size()

**What is the date range of our collection?** GEE has a set of methods called "Reducers" which do a range of functions, such as get the sum or average value of a collection.

In [8]:
imgrange = dmsp.reduceColumns(ee.Reducer.minMax(), ['system:time_start'])
start = ee.Date(imgrange.get('min')).getInfo()['value']
end = ee.Date(imgrange.get('max')).getInfo()['value']
print(f'Date range: {start, end}')

Date range: (694224000000, 1356998400000)


## Date Conversions

Tracking time in a world with many timezones is incredibly complicated. And you can imagine the nightmare scenario if applications (like airplane navigation systems) relied on arbitrary or inconsistent methods of tracking time. Therefore, software applications (including those that preprocessed our DMSP-OLS files as well as GEE) use what’s called **Unix time**, which is what we see here.

To convert these Unix time values to readable dates, we'll:
1. Divide by 1000 to convert to seconds
2. Use `datetime` library to convert to datetime object
3. Convert this datetime object to a string that we can read with the pattern: Year (%Y) - Month (%m) - Day (%d) - Hour (%H): Minute (%M):Second (%S)

In [9]:
# Convert date
start_p = datetime.utcfromtimestamp(start / 1000).strftime('%Y-%m-%d %H:%M:%S')
end_p = datetime.utcfromtimestamp(end / 1000).strftime('%Y-%m-%d %H:%M:%S')
print(f'Date range: {start_p, end_p}')

Date range: ('1992-01-01 00:00:00', '2013-01-01 00:00:00')


In [10]:
# Create a function
def get_date_range(img_collection):
    imgrange = img_collection.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
    start = ee.Date(imgrange.get('min')).getInfo()['value']
    end = ee.Date(imgrange.get('max')).getInfo()['value']

    start = datetime.utcfromtimestamp(start/1000).strftime('%Y-%m-%d %H:%M:%S')
    end = datetime.utcfromtimestamp(end/1000).strftime('%Y-%m-%d %H:%M:%S')
    print(f"Date range: {start, end}")

# Get DMSP-OLS annual composite for 1996

In [11]:
dmsp1996 = ee.Image('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F121996')

map2 = geemap.Map(center = [center_lat, center_lon], zoom = zoom)
map2.add_basemap('SATELLITE')

map2.addLayer(dmsp1996.mask(dmsp1996), {}, 'DMSP-OLS 1996', opacity = 0.75)
map2.addLayerControl()
map2

Map(center=[38.9072, -77.0369], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

# Add a 2nd annual composite from 2010 and create a slider panel

In [12]:
dmsp2010 = ee.Image('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F182010')

map2.addLayer(dmsp2010.mask(dmsp2010), {}, 'DMSP-OLS 2010', opacity = 0.75)

In [13]:
map3 = geemap.Map(center = [center_lat, center_lon], zoom = zoom)
map3.add_basemap('SATELLITE')

# Generate tile layers
dmsp1996_tile = geemap.ee_tile_layer(dmsp1996.mask(dmsp1996), {}, 'DMSP-OLS 1996', opacity = 0.75)
dmsp2010_tile = geemap.ee_tile_layer(dmsp2010.mask(dmsp2010), {}, 'DMSP-OLS 2010', opacity = 0.75)

# Create split map
map3.split_map(left_layer = dmsp1996_tile, right_layer = dmsp2010_tile)
map3

Map(center=[38.9072, -77.0369], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…